# Computer Assignment 1, Search
## The perpose of this project is to provide a solution to the snake game problem using informed and uninformed search algorithms.
### The algorithms used in the project are: BFS, IDS and A*

## Reading Data
The input data is as follows:
1 - Game screen size
2 - The initial coordinates of the snake on the page
3 - Number of seeds
4 - The coordinates of each seed with its score

In [1]:
def read_test_file(path):
    with open(path) as file:
        lines = [line.rstrip().split(',') for line in file]
    return lines

## Classes used in the project

### 1 - Unit
This class represents each of the cells in the game table.

In [5]:
class Unit:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def get_right(self):
        return Unit(self.x, (self.y + 1)% play_ground.length)

    def get_left(self):
        return Unit(self.x, (self.y - 1)% play_ground.length)

    def get_up(self):
        return Unit((self.x - 1)% play_ground.width, self.y)

    def get_down(self):
        return Unit((self.x + 1)% play_ground.width, self.y)
    
    def __hash__(self):
        return hash((self.x, self.y))
        
    def distance_to(self, other):
        return abs(self.x - other.x) + abs(self.y - other.y)

### 2 - Play_ground
This class represents the playground.

In [3]:
class Play_ground:
    def __init__(self, width, length):
        self.width = width
        self.length = length

### 3 - Seed

In [6]:
class Seed:
    def __init__(self, x, y, score):
        self.score = score
        self.home = Unit(x,y)
    
    def __hash__(self):
        return hash((self.score, self.home))

### 4 - Snake

In [7]:
class Snake:
    def __init__(self, head_x, head_y, body = []):
        self.head = Unit(head_x, head_y)
        self.body = body

    def __hash__(self):
        return hash((self.head, hash(frozenset([unit for unit in self.body]))))
    
    def can_move(self, direction):
        if direction == 'R':
            next_unit = self.head.get_right()
        elif direction == 'L':
            next_unit = self.head.get_left()
        elif direction == 'U':
            next_unit = self.head.get_up()
        else:
            next_unit = self.head.get_down()

        if len(self.body) == 1:
            if self.body[0].x == next_unit.x and self.body[0].y == next_unit.y:
                return False
        else:
            for i in range(len(self.body) - 1):
                if self.body[i].x == next_unit.x and self.body[i].y == next_unit.y:
                    return False
        return True
    
    def move(self, direction, seeds):
        next_unit = None
        if direction == 'R':
            next_unit = self.head.get_right()
        elif direction == 'L':
            next_unit = self.head.get_left()
        elif direction == 'U':
            next_unit = self.head.get_up()
        else:
            next_unit = self.head.get_down()

        new_body = [Unit(unit.x, unit.y) for unit in self.body]
        new_snake = Snake(self.head.x, self.head.y, new_body)
        new_seeds = set()

        for seed in seeds:
            new_seeds.add(Seed(seed.home.x, seed.home.y, seed.score))

        can_eat = False
        

        for seed in new_seeds:
            if seed.home.x == next_unit.x and seed.home.y == next_unit.y:
                if(seed.score == 1):
                    new_seeds.remove(seed)
                else:
                    seed.score = seed.score - 1
                can_eat = True
                break
        
        new_snake.head = next_unit
        new_snake.body.insert(0, self.head)

        if not can_eat:
            if len(new_snake.body) != 0:
                new_snake.body.pop()
        
        return new_snake, new_seeds

### 5 - State
This class models the different modes of our game world, what the game snake is like in this mode and what each of the seeds is like.

In [8]:
class State:
    def __init__(self, snake, seeds):
        self.snake = snake
        self.seeds = seeds

    def is_goal(self):
        if len(self.seeds) == 0:
            return True
        return False
    
    def __hash__(self):
        return hash((self.snake, hash(frozenset(self.seeds))))
    
    def get_nexts(self):
        nexts = []
        if self.snake.can_move('R'):
            new_snake, new_seeds = self.snake.move('R', self.seeds)
            nexts.append(State(new_snake, new_seeds))
        if self.snake.can_move('L'):
            new_snake, new_seeds = self.snake.move('L', self.seeds)
            nexts.append(State(new_snake, new_seeds))
        if self.snake.can_move('U'):
            new_snake, new_seeds = self.snake.move('U', self.seeds)
            nexts.append(State(new_snake, new_seeds))
        if self.snake.can_move('D'):
            new_snake, new_seeds = self.snake.move('D', self.seeds)
            nexts.append(State(new_snake, new_seeds))
        
        return nexts

### 6 - Node
This class is to show the path that is taken to reach the answer.

In [180]:
class Node:
    def __init__(self, state, parent, path_cost):
        self.state = state
        self.parent = parent
        self.path_cost = path_cost
    
    def get_children(self):
        return [Node(child_state, self, self.path_cost + 1) for child_state in self.state.get_nexts()]
    
    def hash_without_cost(self):
        return(hash(self.state))
    
    def hash_with_cost(self):
        return(hash((self.state, self.path_cost)))
    
    def __lt__(self, other):
        return self.f() < other.f()
    
    def h(self):
        return len(self.state.seeds)
    
#     def h(self):
#         remain_seeds = 0
#         seeds = self.state.seeds
#         for seed in seeds:
#             remain_seeds = remain_seeds + seed.score
#         return remain_seeds 
    
    def g(self):
        return self.path_cost
    
    def f(self):
        return self.g() + self.h()
    
    def get_path(self):
        head_units = []
        if(self.parent):
            head_units = self.parent.get_path()
        head_units.append(self.state.snake.head)
        return head_units

    def show(self):
        if(self.parent):
             self.parent.show()
        snake_units = []
        for i in range(len(self.state.snake.body)-1, -1):
            snake_units.append(self.state.snake.body[i])
        snake_units.append(self.state.snake.head)
        print('Snake :')
        for unit in snake_units:
            print(unit.x, unit.y)
        print('Seeds :')
        for seed in self.state.seeds:
            if(seed.score > 0):
                print(seed.home.x, seed.home.y, seed.score)
        print('- - - - - - - - - - - - - ')

## Initial objects

In [183]:
test_file_path = './tests/test2.txt'
test_lines = read_test_file(test_file_path)

play_ground = Play_ground(int(test_lines[0][0]), int(test_lines[0][1]))

initial_seed_numbers = int(test_lines[2][0])
initial_seeds = set()
for i in range(initial_seed_numbers):
    initial_seeds.add(Seed(int(test_lines[3+i][0]), int(test_lines[3+i][1]), int(test_lines[3+i][2])))

initial_snake = Snake(int(test_lines[1][0]), int(test_lines[1][1]))

## Show directions
This function shows where the snake has gone in each step by taking path nodes.

In [12]:
def showDirection(heads):
    for i in range(len(heads) - 1):
        if heads[i].get_right().x == heads[i+1].x and heads[i].get_right().y == heads[i+1].y:
            print('R', end = ' ')
        elif heads[i].get_left().x == heads[i+1].x and heads[i].get_left().y == heads[i+1].y:
            print('L', end = ' ')
        elif heads[i].get_up().x == heads[i+1].x and heads[i].get_up().y == heads[i+1].y:
            print('U', end = ' ')
        elif heads[i].get_down().x == heads[i+1].x and heads[i].get_down().y == heads[i+1].y:
            print('D', end = ' ')

## How to model the problem
### Initial state: 
This is the initial state of the problem. The length of the snake is one unit and its coordinates are taken as input. The seeds are also according to the input of the problem.
### Goal state:
It is a situation that we want to achieve and it happens when the snake has eaten all the seeds on the playground.
### Action:
The snake can move in any of the four directions that do not touch its body, and if there is a seed in the house where the snake's head goes, it will eat it and one of the seed scores will be reduced and one will be added to the snake's length.
### Path cost: 
Each move costs 1.

## BFS
In this algorithm, we move forward level by level.
In this way, we first add a node with initial state to the frontier set and add the same to the explored set, and then add all its children to the frontier set if they are not in frontier set or explored set. Then, in order to move forward level by level, we add the first member of frontier set to explored set and continue the previous steps. Whenever the node selected for expansion meets the conditions of the goal state, we get the answer.

In [79]:
from time import time
def bfs():
    start = time()
    
    initial_state = State(initial_snake, initial_seeds)
    initial_node = Node(initial_state, None, 0)
    
    states_num = 1
    seperate_states_num = 1

    frontier = [initial_node]
    frontier_hash = set()
    frontier_hash.add(initial_node.hash_without_cost())
    explored_hash = set()

    if initial_state.is_goal():
        return initial_node, time() - start, states_num, seperate_states_num
    
    while True :
        if len(frontier) == 0: 
            return None, time() - start, states_num, seperate_states_num
        node = frontier.pop(0)
        if node.state.is_goal():
                return node, time() - start, states_num, seperate_states_num
        explored_hash.add(node.hash_without_cost())
        for child in node.get_children():
            states_num += 1
            if child.hash_without_cost() in frontier_hash or child.hash_without_cost() in explored_hash:
                continue
            seperate_states_num += 1
            frontier.append(child)
            frontier_hash.add(child.hash_without_cost())

#### Test1

In [21]:
node, time, states_num, seperate_states_num = bfs()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

0.27637481689453125
12
13018
5747
L D R R D D R R D R D D 

#### Test2

In [52]:
node, time, states_num, seperate_states_num = bfs()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

3.6240060329437256
15
116674
51750
R U L L D L U U U U L L L L U 

#### Test3

In [80]:
node, time, states_num, seperate_states_num = bfs()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

19.609614849090576
25
536081
234039
R U R D D D R R D R R D D R U R R D L L L L L U U 

## IDS
In this algorithm, we test the DFS with different depth limits, and if we do not get the answer, we increase the depth by one again.
To implement the DFS algorithm itself, we start with a node that has initial state. We add the children of the node to the frontier set if they are not in the frontier and explored sets, and also if it's path_cost is not more than depth. Each time we select the last element in the F set to select the new node.

In [117]:
from time import time
def dfs(depth):
    initial_state = State(initial_snake, initial_seeds)
    initial_node = Node(initial_state, None, 0)
    
    states_num = 1
    seperate_states_num = 1
    
    frontier = [initial_node]
    explored_hash = set()
    frontier_hash = set()
    frontier_hash.add(initial_node.hash_with_cost())

    if initial_state.is_goal():
        return initial_node, states_num, seperate_states_num
    
    while True :
        if len(frontier) == 0: 
            return None, states_num, seperate_states_num
        node = frontier.pop()
        explored_hash.add(node.hash_with_cost())

        if node.state.is_goal():
                return node, states_num, seperate_states_num
        for child in node.get_children():
            states_num += 1
            if child.hash_with_cost() in frontier_hash or child.hash_with_cost() in explored_hash or child.path_cost > depth:
                continue
            seperate_states_num += 1
            frontier.append(child)
            frontier_hash.add(child.hash_with_cost())

In [118]:
def ids():
    depth = 0
    start = time()
    all_states_num = 0
    all_seperate_states_num = 0
    while True:
        result, states_num, seperate_states_num = dfs(depth)
        all_states_num += states_num
        all_seperate_states_num += seperate_states_num
        if result:
            return result, time() - start, all_states_num, all_seperate_states_num
        depth += 1
    return None, time() - start, all_states_num, all_seperate_states_num

#### Test1

In [110]:
node, time, states_num, seperate_states_num = ids()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

1.7015631198883057
12
90962
29974
D L D D R R R D R D D R 

#### Test2

In [114]:
node, time, states_num, seperate_states_num = ids()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

15.80132794380188
15
786582
264268
U R D L L U U U U L U L L L L 

#### Test3

In [119]:
node, time, states_num, seperate_states_num = ids()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

97.26926112174988
25
4849363
1641932
U R D D D R D R R D D R R R U R R D L L L U U L L 

## A*
This type of search is informed in the sense that each time we select a node, the sum of the distance traveled so far and the prediction we have of its future direction is the smallest.


In [184]:
from heapq import heappush, heappop
from time import time

def a_star():
    start = time()
    
    initial_state = State(initial_snake, initial_seeds)
    initial_node = Node(initial_state, None, 0)
    
    states_num = 1
    seperate_states_num = 1
    
    frontier = []
    explored_hash = set()
    heappush(frontier, initial_node)
    frontier_hash = dict()
    frontier_hash[initial_node.hash_without_cost()] = initial_node.path_cost

    if initial_state.is_goal():
        return initial_node, states_num, seperate_states_num
    
    while True :
        if len(frontier) == 0: 
            return None, time() - start, states_num, seperate_states_num
        node = heappop(frontier)
        explored_hash.add(node.hash_without_cost())
        if node.state.is_goal():
                return node, time() - start, states_num, seperate_states_num
        for child in node.get_children():
            states_num += 1
            child_hash = child.hash_without_cost()
            if child_hash in explored_hash or (child_hash in frontier_hash and child.path_cost >= frontier_hash[child_hash]):
                continue
            seperate_states_num += 1
            heappush(frontier, child)
            frontier_hash[child.hash_without_cost()] = child.path_cost

### first heuristic
Here we consider the h function to return the total number of seeds.
This is a admissible heuristic because with each move of the snake, in the best case, a seed is eaten, so the snake must move at least the number of remaining seeds to reach the goal state, and the heuristic value is always less than the real value.
This is a consistent heuristic because in each move from one node to another node, the seed numbers does not decrease or decreases by one unit, so in any case between two neighbor nodes, h(parent) is more than or equal to h(child).

#### Test1

In [32]:
node, time, states_num, seperate_states_num = a_star()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

0.12101244926452637
12
4495
2500
D L L U U L U L L L U U 

#### Test2

In [185]:
node, time, states_num, seperate_states_num = a_star()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

1.4019811153411865
15
41505
22004
U R D L L U U U U L L U L L L 

#### Test3

In [182]:
node, time, states_num, seperate_states_num = a_star()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

9.830609321594238
25
239891
115055
U R D D D R R R D R D D R R R R U L L D L L U L U 

### second heuristic
Here we consider the h function to return the total score of seeds remaining for each node.
This is a admissible heuristic because with each move of the snake, in the best case, a one-point seed is eaten, so the snake must move at least the number of remaining seeds to reach the goal state, and the heuristic value is always less than the real value.
This is a consistent heuristic because in each move from one node to another node, the seed score does not decrease or decreases by one unit, so in any case between two neighbor nodes, h(parent) is more than or equal to h(child).

#### Test1

In [167]:
node, time, states_num, seperate_states_num = a_star()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

0.1319882869720459
12
4235
2651
D L U U L L U L L L U U 

#### Test2

In [170]:
node, time, states_num, seperate_states_num = a_star()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

2.079922914505005
15
41505
22004
U R D L L U U U U L L U L L L 

#### Test3

In [179]:
node, time, states_num, seperate_states_num = a_star()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

9.807828903198242
25
199531
99034
U R D D D R D R R R R D D R U R R D L L L U L U L 

## Weighted A*

When the heuristic is admissble, we are sure that its value is less than the real value. By multiplying it, the heuristic value gets closer to the real number and the result is obtained in a shorter time.
But if this multiplication causes the answer to no longer be admissble, it may not give us the optimal path.

#### alpha = 1.8

#### Test1

In [127]:
node, time, states_num, seperate_states_num = a_star()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

0.10935211181640625
12
3805
2196
D L U U L L U L L L U U 

#### Test2

In [133]:
node, time, states_num, seperate_states_num = a_star()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

0.7829029560089111
15
24048
13817
R U L D L U U U U L L L L U L 

#### Test3

In [124]:
node, time, states_num, seperate_states_num = a_star()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

6.9806060791015625
25
172466
84486
U R D D D R R D R R D D R R U R R D L L L L U U L 

#### alpha = 4.8

#### Test1

In [144]:
node, time, states_num, seperate_states_num = a_star()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

0.11131906509399414
16
4217
1873
U L L L U L L U U L D D L U U R 

#### Test2

In [141]:
node, time, states_num, seperate_states_num = a_star()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

0.12711381912231445
17
3283
2300
U L D R R U L L U U L U L L L U L 

#### Test3

In [147]:
node, time, states_num, seperate_states_num = a_star()
print(time)
heads = node.get_path()
print(node.path_cost)
print(states_num)
print(seperate_states_num)
showDirection(heads)

1.3445119857788086
26
39208
21576
R U R D D D R R D R U L D D D R R R U R R D D L U L 

## Differences

#### Completness and optimality
All of this algorithms are complete and optimal.

#### Time complexity
BFS runs in O($ b^d $) where d is depth of solution.

IDS runs in O($ b^d $) too but it's a bit slower than BFS because it runs DFS sunbroutines many times.

A* runs in O(Number of nodes for which f(n) ≤ C* (exponential))

#### Space Complexity
BFS gets O($ b^d $) space because it saves all the visited and frontier nodes till goal node.

IDS gets O( bd ) space because it just saves frontier nodes for one branch. 

A* get exponential space. 

## Result table

#### Test1

| algorithm | path cost | path | number of states visited | number of separate states visited | time
| --- | --- | --- | --- | --- | --- |
| BFS | 12 | L D R R D D R R D R D D | 13018 | 5747 | 0.31 |
| IDS | 12 | D L D D R R R D R D D R | 90962 | 29974 | 1.68 |
| A* h1 | 12 | D L L U U L U L L L U U | 4495 | 2500 | 0.12 |
| A* h2 | 12 | D L U U L L U L L L U U  | 4235 | 2651 | 0.13 |
| weighted A* 1 | 12 | D L U U L L U L L L U U | 3805 | 2196 | 0.11 |
| weighted A* 2 | 16 | U L L L U L L U U L D D L U U R | 4217 | 1873 | 0.115 |


#### Test2

| algorithm | path cost | path | number of states visited | number of separate states visited | time
| --- | --- | --- | --- | --- | --- |
| BFS | 15 | R U L L D L U U U U L L L L U | 116674 | 51750 | 3.45 |
| IDS | 15 | U R D L L U U U U L U L L L L | 786582 | 264268 | 15.64 |
| A* h1 | 15 | U R D L L U U U U L L U L L L | 41505 | 22004 | 1.5 |
| A* h2 | 15 | U R D L L U U U U L L U L L L | 41505 | 22004 | 2.1 |
| weighted A* 1 | 12 | D L U U L L U L L L U U | 3805 | 2196 | 0.79 |
| weighted A* 2 | 17 | U L D R R U L L U U L U L L L U L | 3283 | 2300 | 0.13 |


#### Test3

| algorithm | path cost | path | number of states visited | number of separate states visited | time
| --- | --- | --- | --- | --- | --- |
| BFS | 25 | R U R D D D R R D R R D D R U R R D L L L L L U U | 536081 | 234039 | 19.34 |
| IDS | 25 | U R D D D R D R R D D R R R U R R D L L L U U L L | 4849363 | 1641932 | 99.02 |
| A* h1 | 25 | U R D D D R R R D R D D R R R R U L L D L L U L U  | 239891 | 115055 | 9.82 |
| A* h2 | 25 | U R D D D R D R R R R D D R U R R D L L L U L U L | 199531 | 99034 | 9.83 |
| weighted A* 1 | 25 | U R D D D R R D R R D D R R U R R D L L L L U U L | 172466 | 84486 | 6.8 |
| weighted A* 2 | 26 | R U R D D D R R D R U L D D D R R R U R R D D L U L | 39208 | 21576 | 1.36 |